In [6]:
from chain import chain_with_history

In [7]:
response = chain_with_history.invoke("What is 9 *121321321")
print(response)

In [5]:
from chain import get_session_history

print(get_session_history())

Human: My name is Andriy
AI: Hello Andriy! How can I assist you today?
Human: My name is Andriy
AI: Hello Andriy! It seems you repeated your name. How can I assist you today?
Human: What is my name?
AI: Your name is Andriy. How can I assist you today?


In [3]:
from langchain.chat_models.base import init_chat_model

# from langchain_openai.chat_models.base import ChatOpenAI

llm = init_chat_model(model="gpt-4o-mini", model_provider="openai")
# llm = ChatOpenAI(model_name="gpt-4o-mini")
response = llm.invoke("Hello, how are you?")

print(response.content)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [5]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain.prompts import PromptTemplate
from os import getenv
from dotenv import load_dotenv

load_dotenv()

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="anthropic/claude-3.5-sonnet",
)


question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
reposne = llm.invoke(question)
print(response.content)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
